### Import Libraries

In [1]:
import pandas as pd
import numpy as np
import pathlib

from sklearn.metrics import f1_score
from joblib import load

import sys
sys.path.append("../utils")
from split_utils import get_features_data
from train_utils import get_dataset, get_X_y_data
from evaluate_utils import evaluate_model_cm, evaluate_model_score

### Load necessary data

In [2]:
# load features data from indexes and features dataframe
data_split_path = pathlib.Path("../1.split_data/indexes/data_split_indexes.tsv")
data_split_indexes = pd.read_csv(data_split_path, sep="\t", index_col=0)
features_dataframe_path = pathlib.Path("../0.download_data/data/training_data.csv.gz")
features_dataframe = get_features_data(features_dataframe_path)

### Get Each Model Predictions on Each Dataset

In [4]:
# directory to load the models from
models_dir = pathlib.Path("../2.train_model/models/")

# use a list to keep track of scores in tidy long format for each model and dataset combination
compiled_predictions = []

# iterate through each model (final model, shuffled baseline model, etc)
for model_path in models_dir.iterdir():
    model = load(model_path)
    model_name = model_path.name.replace("log_reg_","").replace(".joblib","")
    
    # iterate through label datasets (labels correspond to train, test, etc)
    # with nested for loops, we test each model on each dataset(corresponding to a label)
    for label in data_split_indexes["label"].unique():
        print(f"Getting predictions for {model_name} on dataset {label}")
        
        # load dataset (train, test, etc)
        data = get_dataset(features_dataframe, data_split_indexes, label)
        
        # get features and labels dataframes
        X, y = get_X_y_data(data)

        # get predictions from model
        y_pred = model.predict(X)
        
        # create dataframe with predicted phenotypic class, true phenotypic class, and which dataset/model is involved in prediction
        predictions_df = pd.DataFrame(
        {'Phenotypic_Class_Predicted': y,
        'Phenotypic_Class_True': y_pred,
        'data_split': label,
        "shuffled": "shuffled" in model_name
        })
        
        break

Getting predictions for model on dataset test
['Polylobed' 'Polylobed' 'OutOfFocus' 'Polylobed' 'Binuclear' 'Grape'
 'Anaphase' 'MetaphaseAlignment' 'SmallIrregular' 'Interphase' 'Polylobed'
 'Grape' 'ADCCM' 'Grape' 'Polylobed' 'Polylobed' 'Polylobed' 'Elongated'
 'Apoptosis' 'Polylobed' 'Binuclear' 'Apoptosis' 'Polylobed' 'Polylobed'
 'Binuclear' 'Polylobed' 'Binuclear' 'Grape' 'Polylobed' 'Polylobed'
 'MetaphaseAlignment' 'Grape' 'Artefact' 'Grape' 'Polylobed' 'Polylobed'
 'Polylobed' 'Prometaphase' 'Large' 'Binuclear' 'Interphase' 'Interphase'
 'Polylobed' 'Metaphase' 'ADCCM' 'SmallIrregular' 'Apoptosis' 'Polylobed'
 'Apoptosis' 'Polylobed' 'Polylobed' 'Elongated' 'Binuclear' 'Anaphase'
 'Binuclear' 'MetaphaseAlignment' 'Polylobed' 'Grape' 'Metaphase'
 'Interphase' 'Grape' 'Metaphase' 'Polylobed' 'Polylobed' 'Interphase'
 'OutOfFocus' 'Polylobed' 'Grape' 'Grape' 'Elongated' 'Binuclear'
 'Polylobed' 'OutOfFocus' 'Hole' 'OutOfFocus' 'Binuclear' 'Polylobed'
 'Elongated' 'OutOfFocus' 'G